<a href="https://colab.research.google.com/github/MeyerTalon/URAP-ML-Interview-Task/blob/main/legal_identifier_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers huggingface_hub pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is i

In [3]:
from huggingface_hub import notebook_login

# We need to login as we'll upload our model and dataset to the hub.
notebook_login()

In [4]:
from datasets import load_dataset
import pandas as pd
import numpy as np

# Generate train, test, and validation splits.
df = pd.read_csv('company_names_parsed.csv')
df_train = df.sample(frac=0.8, random_state=42)
df_test = df.drop(df_train.index)
df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)

# Load the dataset and push it to Huggingface Hub for access anywhere.
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})
dataset.push_to_hub('TalonMeyer/URAP_interview_task_dataset')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/80 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/483 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/TalonMeyer/URAP_interview_task_dataset/commit/83005d4c8f04937229ddf3f32cd57d949c8ec2da', commit_message='Upload dataset', commit_description='', oid='83005d4c8f04937229ddf3f32cd57d949c8ec2da', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
dataset = dataset.remove_columns(['location', 'base_name'])
dataset = dataset.rename_columns({'legal': 'labels', 'raw': 'text'})

def transform_null_to_one(row):
    # Replace non-null values with 1, and null values with 0
    return 1 if row['labels'] is not None else 0

# Apply the transformation to the dataset
dataset = dataset.map(lambda x: {'labels': transform_null_to_one(x)})

Map:   0%|          | 0/79990 [00:00<?, ? examples/s]

Map:   0%|          | 0/19997 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = 'google-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(componenets: dict) -> dict:
  """
  Tokenizes the components of the dataset.

  Args:
    components: A dictionary containing the components of the dataset.

  Returns:
    A dictionary containing the tokenized components of the dataset.
  """

  return tokenizer(componenets['text'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/79990 [00:00<?, ? examples/s]

Map:   0%|          | 0/19997 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
  """
  Computes the accuracy of the model.

  Args:
    eval_pred: A tuple containing the logits and labels of the model.

  Returns:
    A dictionary containing the accuracy of the model.
  """
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)


In [9]:
training_args = TrainingArguments(output_dir='trainer', eval_strategy='epoch')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    )

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.066700,0.065763,0.988848


In [ ]:
prediction = trainer.predict('Chase Bank Inc').predictions.argmax(axis=1)
prediction

In [ ]:
kwargs = {
    'finetuned_from': model.config._name_or_path,
    'dataset': 'TalonMeyer/URAP_interview_task_dataset',
    'tasks': 'text-classification',
    'tags': ['text-classification'],
}
trainer.save_model()
trainer.push_to_hub('TalonMeyer/bert-base-cased-legal-keyword-identifier', **kwargs)